In [48]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [49]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [50]:
external_data = pd.read_csv(Path("data") / "external_data.csv")

In [51]:
data['date'] = pd.to_datetime(data['date'])
external_data['date'] = pd.to_datetime(external_data['date'])

In [52]:
df = pd.merge(data, external_data, how='inner', on='date')
df

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1700.0,5.0,6.0,2300.0,NaN,NaN,NaN,NaN,NaN,NaN
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1700.0,7.0,6.0,2400.0,NaN,NaN,NaN,NaN,NaN,NaN
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,7.0,2020-09-02 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,12.0,2020-09-03 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,1830.0,5.0,0.0,8000.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165363,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,211.0,2021-09-07 15:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165364,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,156.0,2021-09-09 18:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,1600.0,2.0,3.0,2800.0,6.0,0.0,8000.0,1.0,9.0,2300.0
165365,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,440.0,2021-09-08 06:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165366,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,445.0,2021-09-09 06:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,...,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Preprocessing


### Removing duplicate rows
There are 54 duplicates. We will remove them to reduce bias.

In [53]:
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
num_duplicates

np.int64(54)

In [56]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

### Handling Missing Values

In [13]:
missing_values = df.isnull().sum()
print("Missing values summary:"), missing_values

Missing values summary:


(None,
 counter_id           0
 counter_name         0
 site_id              0
 site_name            0
 bike_count           0
                  ...  
 ctype3          142612
 hnuage3         135304
 nnuage4         163478
 ctype4          161256
 hnuage4         163478
 Length: 70, dtype: int64)

For other missing values, we have 2 options: drop or impute.

We drop columns with >=50% missing values. So we drop 21 columns in total.

In [42]:
df.drop(df.columns[df.isna().mean()>0.5], axis=1, inplace=True)
df.shape

(165314, 49)

We also have different options to impute. We can impute with mean, median, mode, or even use models to predict missing values

## Feature Engineering

### Rescale

Log transform count_bike. We have done this before merging the data. 

### Encode 

#### Encode Time

#### Encode Weather

## Selecting features with ANOVA

# Modelling

Data for modelling

In [37]:
dropped_columns = ['counter_id', 'counter_name', 'site_id', 'site_name', 
                   'bike_count', 'date', 'counter_installation_date', 'coordinates',
                   'counter_technical_id', 'latitude', 'longitude']


In [38]:
Train_data = df.drop(columns=dropped_columns)

In [39]:
X = Train_data.drop(columns=["log_bike_count"])
y = Train_data['log_bike_count']

Train-test split

In [30]:
from sklearn.model_selection import train_test_split
 

Model with Vectorized Table

In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from dirty_cat import TableVectorizer

model = make_pipeline(
    TableVectorizer(), 
    HistGradientBoostingRegressor()
)

model.fit(X_train, y_train)  

ImportError: cannot import name '_parse_version' from 'sklearn.utils.fixes' (/opt/anaconda3/envs/bikes-count/lib/python3.10/site-packages/sklearn/utils/fixes.py)

We obtain the predicted outcome and evaluated the accuracy.

In [57]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 Score): {r2}")

NameError: name 'model' is not defined